# Model constructor.

> Create and tune pytorch model.

In [8]:
#hide
%load_ext autoreload
%autoreload 2

In [11]:
#hide
import torch
import torch.nn as nn

from nbdev.showdoc import show_doc
from IPython.display import Markdown, display

In [12]:
# hide
def print_doc(func_name):
    doc = show_doc(func_name, title_level=4, disp=False)
    display(Markdown(doc))

## ResBlock

In [13]:
#hide
from model_constructor.model_constructor import ResBlock

In [14]:
#hide_input
print_doc(ResBlock)

<h4 id="ResBlock" class="doc_header"><code>class</code> <code>ResBlock</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/model_constructor.py#L25" class="source_link" style="float:right">[source]</a></h4>

> <code>ResBlock</code>(**`expansion`**, **`in_channels`**, **`mid_channels`**, **`stride`**=*`1`*, **`conv_layer`**=*`ConvLayer`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`zero_bn`**=*`True`*, **`bn_1st`**=*`True`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`sa`**=*`False`*, **`sym`**=*`False`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*, **`se_module`**=*`SEModule`*, **`se`**=*`False`*, **`se_reduction`**=*`16`*) :: `Module`

Resnet block

In [27]:
block = ResBlock(1,64,64,sa=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [28]:
block.id_conv

In [30]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = block(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [31]:
block = ResBlock(4,64,64,sa=True, dw=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [32]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = block(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [33]:
block = ResBlock(4,64,64,sa=True, groups=4)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [34]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = block(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [35]:
block = ResBlock(2,64,64,act_fn=nn.LeakyReLU(), bn_1st=False)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [36]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 128, 32, 32)
y = block(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 128, 32, 32]), f"size"

torch.Size([16, 128, 32, 32])


In [37]:
block = ResBlock(2, 64, 64, sa=True, se=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEModule(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (fc_reduce): Linear(in_features=128, out_features=8, bias=True)
        (se_act): ReLU(inplace=True)
        (fc_expand): Linear(in_features=8, out_features=128, bias=True

In [38]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 128, 32, 32)
y = block(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 128, 32, 32]), f"size"

torch.Size([16, 128, 32, 32])


In [39]:
block = ResBlock(2, 32, 64, dw=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (id_conv): Sequential(
    (id_conv): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [40]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = block(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 128, 32, 32]), f"size"

torch.Size([16, 128, 32, 32])


In [41]:
block = ResBlock(2, 32, 64, stride=2, dw=True)
block

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (id_conv): Sequential(
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (id_conv): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_

In [42]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = block(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 128, 16, 16]), f"size"

torch.Size([16, 128, 16, 16])


## Stem, Body, Head

In [23]:
#hide
from model_constructor.model_constructor import _make_body, _make_head, _make_layer, _make_stem

In [24]:
#hide_input
print_doc(_make_layer)

<h4 id="_make_layer" class="doc_header"><code>_make_layer</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/model_constructor.py#L76" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_layer</code>(**`expansion`**, **`in_channels`**, **`out_channels`**, **`blocks`**, **`stride`**, **`sa`**)



In [25]:
#hide_input
print_doc(_make_stem)

<h4 id="_make_stem" class="doc_header"><code>_make_stem</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/model_constructor.py#L63" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_stem</code>()



In [26]:
#hide_input
print_doc(_make_body)

<h4 id="_make_body" class="doc_header"><code>_make_body</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/model_constructor.py#L87" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_body</code>()



In [27]:
#hide_input
print_doc(_make_head)

<h4 id="_make_head" class="doc_header"><code>_make_head</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/model_constructor.py#L97" class="source_link" style="float:right">[source]</a></h4>

> <code>_make_head</code>()



## Model Constructor.

In [28]:
#hide
from model_constructor import ModelConstructor

In [29]:
#hide_input
print_doc(ModelConstructor)

<h4 id="ModelConstructor" class="doc_header"><code>class</code> <code>ModelConstructor</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/model_constructor.py#L104" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelConstructor</code>(**`name`**=*`'MC'`*, **`c_in`**=*`3`*, **`c_out`**=*`1000`*, **`block`**=*`ResBlock`*, **`conv_layer`**=*`ConvLayer`*, **`block_sizes`**=*`[64, 128, 256, 512]`*, **`layers`**=*`[2, 2, 2, 2]`*, **`norm`**=*`BatchNorm2d`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`expansion`**=*`1`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*, **`sa`**=*`False`*, **`se`**=*`False`*, **`se_module`**=*`SEModule`*, **`se_reduction`**=*`16`*, **`bn_1st`**=*`True`*, **`zero_bn`**=*`True`*, **`stem_stride_on`**=*`0`*, **`stem_sizes`**=*`[32, 32, 64]`*, **`stem_pool`**=*`MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)`*, **`stem_bn_end`**=*`False`*, **`_init_cnn`**=*`init_cnn`*, **`_make_stem`**=*`_make_stem`*, **`_make_layer`**=*`_make_layer`*, **`_make_body`**=*`_make_body`*, **`_make_head`**=*`_make_head`*)

Model constructor. As default - xresnet18

In [30]:
mc  = ModelConstructor()
mc

MC constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [31]:
mc._block_sizes

[64, 128, 256, 512]

In [32]:
mc.block_sizes

[64, 64, 128, 256, 512]

In [33]:
mc._block_sizes = [128, 256, 512, 1024]
mc

MC constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [128, 256, 512, 1024]
  layers: [2, 2, 2, 2]

In [34]:
mc.block_sizes

[64, 128, 256, 512, 1024]

In [35]:
#hide
mc  = ModelConstructor(stem_sizes=[3,32,32,64])
mc

MC constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [36]:
mc.block_sizes

[64, 64, 128, 256, 512]

In [37]:
model = ModelConstructor()

In [38]:
#hide
# model.block_sizes = [64, 128, 256, 512] # wrong way --> use _block_sizes
# model

In [39]:
#collapse_output
mc.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [40]:
#collapse_output
mc.stem_stride_on = 1
mc.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [41]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mc.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


/home/jzz/anaconda3/envs/mc_dev/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [42]:
mc.bn_1st = False

In [43]:
mc.act_fn =nn.LeakyReLU(inplace=True)

In [44]:
mc.sa = True
mc.se = True

In [45]:
#collapse_output
mc.body.l_1

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEModule(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=128, out_features=8, bias=True)
          (se_act): ReLU(inplace=True)
          (fc_expand): Linear(in_features=8, out_features=128, bias=True)
          (se_gate): Sigmoid()
        )
      )
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): ConvLay

In [46]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = mc.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [47]:
#hide
mc.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEModule(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=64, out_features=4, bias=True)
          (se_act): ReLU(inplace=True)
          (fc_expand): Linear(in_features=4, out_features=64, bias=True)
          (se_gate): Sigmoid()
        )
      )
    )
    (pool): Identity()
    (idconv): Identity()
    (act_fn): LeakyReLU(negative_slope

In [48]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = mc.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [49]:
#hide
mc.groups = 4
mc.expansion = 4
mc.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEModule(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=256, out_features=

In [50]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = mc.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [51]:
#hide
mc.groups = 1
mc.dw = True
mc.expansion = 4
mc.body.l_0

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_2): ConvLayer(
        (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SEModule(
        (squeeze): AdaptiveAvgPool2d(output_size=1)
        (excitation): Sequential(
          (fc_reduce): Linear(in_features=256, out_features

In [52]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = mc.body.l_0(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 256, 32, 32]), f"size"

torch.Size([16, 256, 32, 32])


In [53]:
#hide
mc.groups = 1
mc.dw = 0
mc.expansion = 1

## end
model_constructor
by ayasyrev